# **Stable Diffusion - LoRA**

# I. Installation

In [1]:
# @title ## Install Dependencies
import os
import zipfile
import shutil
import time
from subprocess import getoutput
from IPython.utils import capture
from run_training import train_model

# Model Training 



In [2]:
# @title ## Model Config
# Change the key to:
# - living
# - bedroom
# - dining
import json

# Uncomment the config file lines for the respective rooms
config_path = './train/LoRA/config/living-config.json'
# config_path = './train/LoRA/config/bedroom-config.json'
# config_path = './train/LoRA/config/dining-config.json'

with open(config_path) as json_file:
    configs = json.load(json_file)

In [4]:
#@title ## Start Training
# Replace invalid config params
if 'stop_text_encoder_training' in configs:
    configs['stop_text_encoder_training_pct'] = configs.pop('stop_text_encoder_training')

train_model(headless=True, print_only=False, **configs)

01:41:54-426821 INFO     Start training LoRA Standard ...

01:41:54-440772 INFO     Checking for duplicate image filenames in training data directory...

01:41:54-447276 INFO     Valid image folder names found in:                                                        
                         C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/SystemCode/src/main/t
                         rain/LoRA/train_data

01:41:54-450297 INFO     Headless mode, skipping verification if model already exist... if model already exist it  
                         will be overwritten...

01:41:54-455363 INFO     Folder 10_living: 160 images found

...

01:41:54-477628 INFO     max_train_steps (1600 / 1 / 4 * 1 * 1) = 400

01:41:54-483059 INFO     stop_text_encoder_training = 0

01:41:54-486466 INFO     lr_warmup_steps = 40

01:41:54-491107 INFO     Saving training config to                                                                 
                         C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/SystemCode/src/main/t
                         rain/LoRA/output\roomifai_living_20231024-014154.json...

01:41:54-497128 INFO     accelerate launch --num_cpu_threads_per_process=2 "./train_network.py" --enable_bucket    
                         --min_bucket_reso=256 --max_bucket_reso=2048                                              
                         --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"                          
                         --train_data_dir="C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/Sys
                         temCode/src/main/train/LoRA/train_data" --resolution="512,512"                            
                         --output_dir="C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/SystemC
                         ode/src/main/train/LoRA/output"                                                           
                         --logging_dir="C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/System
                         Code/src/main/train/LoRA/logs" --network_alpha="1" --save_model_as=safetensors            
                         --network_module=networks.lora --text_encoder_lr=5e-05 --unet_lr=0.0001 --network_dim=8   
                         --gradient_accumulation_steps=4 --output_name="roomifai_living"                           
                         --lr_scheduler_num_cycles="1" --no_half_vae --learning_rate="0.0001"                      
                         --lr_scheduler="cosine" --lr_warmup_steps="40" --train_batch_size="1"                     
                         --max_train_steps="400" --save_every_n_epochs="1" --mixed_precision="fp16"                
                         --save_precision="fp16" --caption_extension=".txt" --cache_latents                        
                         --optimizer_type="AdamW8bit" --max_data_loader_n_workers="0" --bucket_reso_steps=64       
                         --save_every_n_steps="100" --bucket_no_upscale --noise_offset=0.0